In [1]:
import torch
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import scipy.io as scio
import random


In [2]:
X = []  # 定义图像名称
Y = []  # 定义图像分类类标

with open('fc_list.txt','r') as f:
    data=f.readlines()
data=[item.replace('\n','').split('\t') for item in data]
random.seed(2022)
random.shuffle(data)
result_index_matrix=torch.zeros(5,3,3)

k_fold=5
for kk in range(k_fold):
    k_fold_len = int(len(data)//k_fold)
    test_data = data[k_fold_len*kk:k_fold_len*(kk+1)]
    train_data = data[:k_fold_len*kk] + data[k_fold_len*(kk+1):]

    X_train=[x[0] for x in train_data]
    y_train=[x[1] for x in train_data]
    X_test=[x[0] for x in test_data]
    y_test=[x[1] for x in test_data]
    X_train=np.array(X_train)
    y_train=np.array(y_train)
    X_test=np.array(X_test)
    y_test=np.array(y_test)

    XX_train = []
    for imagepath in X_train:
        # temp_image=h5py.File(imagepath,'r')
        temp_image=scio.loadmat(imagepath)['fc']
        XX_train.append(temp_image.flatten())  #reshape into 1D array

    # 测试集
    XX_test = []
    for imagepath in X_test:
        # temp_image=h5py.File(imagepath,'r')
        temp_image=scio.loadmat(imagepath)['fc']
        XX_test.append(temp_image.flatten())

    # 第三步 基于支持向量机的图像分类处理
    # 0.5
    # 常见核函数‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’
    clf = SVC().fit(XX_train, y_train)
    clf = SVC(kernel="linear").fit(XX_train, y_train)
    predictions_labels = clf.predict(XX_test)

    c_matrix=confusion_matrix(y_test,predictions_labels)
    for idm in range(3):
        result_index_matrix[kk][0][idm]=c_matrix[idm][idm]/np.sum(c_matrix[:,idm])
        result_index_matrix[kk][1][idm]=c_matrix[idm][idm]/np.sum(c_matrix[idm,:])
        result_index_matrix[kk][2][idm]=(np.sum(c_matrix)-np.sum(c_matrix[:,idm])-np.sum(c_matrix[idm,:])+c_matrix[idm][idm])/(np.sum(c_matrix)-np.sum(c_matrix[idm,:])) 

    print(classification_report(y_test, predictions_labels))
    print(result_index_matrix[kk])
mean_matrix=result_index_matrix.sum(dim=0)/5
print(mean_matrix)
print(mean_matrix.sum(dim=1)/3)

                        precision    recall  f1-score   support

birth_below_scan_above       0.78      0.92      0.84        50
      birth_scan_above       0.92      0.72      0.81        47
      birth_scan_below       0.96      0.98      0.97        56

              accuracy                           0.88       153
             macro avg       0.89      0.88      0.88       153
          weighted avg       0.89      0.88      0.88       153

tensor([[0.7797, 0.9189, 0.9649],
        [0.9200, 0.7234, 0.9821],
        [0.8738, 0.9717, 0.9794]])
                        precision    recall  f1-score   support

birth_below_scan_above       0.50      0.83      0.62        36
      birth_scan_above       0.82      0.53      0.65        60
      birth_scan_below       0.96      0.91      0.94        57

              accuracy                           0.75       153
             macro avg       0.76      0.76      0.74       153
          weighted avg       0.80      0.75      0.75       